# Assignment 3

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [2]:
#nltk.download('wordnet')
#nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

NameError: name 'nltk' is not defined

In [290]:

lemmatizer = WordNetLemmatizer() 

In [4]:
#Location of training and development dataset
data_loc = 'C:/Users/Yorran Reurich/OneDrive/PC_Home/Master/Applied Text Mining/Data/'
train_data_name = 'SEM-2012-SharedTask-CD-SCO-training-simple.v2.txt'
development_data_name = 'SEM-2012-SharedTask-CD-SCO-dev-simple.v2.txt'
cols = ['text', 'sentence_no', 'word_no', 'token', 'neg_cue']

In [5]:
def read_data(data_location, file_name, col_names):
    df = pd.read_csv(data_location + file_name, sep='\t', names=cols)
    return df
    

In [15]:
train_data = read_data(data_loc, train_data_name, cols)
dev_data = read_data(data_loc, development_data_name, cols)
train_data.head(50)

,text,sentence_no,word_no,token,neg_cue
0,baskervilles01,0,0,Chapter,O
1,baskervilles01,0,1,1.,O
2,baskervilles01,0,2,Mr.,O
3,baskervilles01,0,3,Sherlock,O
4,baskervilles01,0,4,Holmes,O
5,baskervilles01,1,0,Mr.,O
6,baskervilles01,1,1,Sherlock,O
7,baskervilles01,1,2,Holmes,O
8,baskervilles01,1,3,",",O
9,baskervilles01,1,4,who,O


In [10]:
max(train_data['sentence_no'])

398

### Lemmatization

In [295]:
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()

To make the lemmatization work, it is crucial to first get the right POS. Which can be done by the nltk.pos_tag function
The POS is not in the current notation as the pos tag that is required by wordnet to find the correct lemma.
The following function will map the pos to a usable pos-tag

In [296]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    true_pos = nltk.pos_tag([word])[0][1]
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN), true_pos

In [297]:
def contains_prefix(word):
    #List can and probably should be extended
    prefix_list = ['in', 'un', 'im', 'il', 'dis']
    if word.startswith(tuple(prefix_list)):
            return True
    else:
        return False



In [298]:
def contains_suffix(word):
    #List can and probably should be extended
    suffix_list = ['less']
    if word.endswith(tuple(prefix_list)):
            return True
    else:
        return False


In [299]:
def add_features(data):
    #Initiate collumns for lemma and pos
    
    data['lemma'] = 0
    data['pos'] = 0
    data['hasPrefix'] = 0
    data['hasSuffix'] = 0

    for i in range(len(data)):
        current_token = data['token'][i]
        pos, orig_pos = get_wordnet_pos(current_token)
        lemma = lemmatizer.lemmatize(current_token, pos)
        
        #Save POS, lemma, prefix in dataframe
        data['pos'][i] = orig_pos
        data['lemma'][i] = lemma
        data['hasPrefix'][i] = contains_prefix(current_token) #CHECK IF WORKS
        data['hasSuffix'][i] = contains_suffix(current_token) #CHECK IF WORKS
            
    return data


In [300]:
train_data = add_features(train_data)
dev_data = add_features(dev_data)


In [301]:
train_data.loc[train_data['token'] != train_data['lemma']]

,text,sentence_no,word_no,token,neg_cue,lemma,pos,hasPrefix,hasSuffix
10,baskervilles01,1,5,was,O,be,VBD,False,False
16,baskervilles01,1,11,mornings,O,morning,NNS,False,False
23,baskervilles01,1,18,occasions,O,occasion,NNS,False,False
26,baskervilles01,1,21,was,O,be,VBD,False,False
31,baskervilles01,1,26,was,O,be,VBD,False,False
...,...,...,...,...,...,...,...,...,...
65420,baskervilles14,270,32,lying,O,lie,VBG,False,False
65432,baskervilles14,270,44,mottled,O,mottle,VBN,False,False
65438,baskervilles14,270,50,stretched,O,stretch,VBN,False,False
65442,baskervilles14,270,54,merged,O,merge,VBN,False,False


Prefix detection: (in-, un-, im-, il-, dis-)

In [302]:
train_data.iloc[22]

text           baskervilles01
sentence_no                 1
word_no                    17
token              infrequent
neg_cue                 B-NEG
lemma              infrequent
pos                        NN
hasPrefix                True
hasSuffix               False
Name: 22, dtype: object

In [303]:
#List can and probably should be extended
test = train_data['token'][22]
prefix_list = ['in', 'un', 'im', 'il', 'dis']
test.startswith(tuple(prefix_list))


True